In [44]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, Point

# Creating Grid
We have a grid for Maize, but it is an incomplete grid, the steps below take that grid and add missing points.

In [45]:
maize_grid = gpd.read_file('./unsynced-data/grid-maize.geojson')
maize_grid = maize_grid.to_crs(epsg=4326)
maize_grid

,lat,lon,Exp.,geometry
0,36.75,5.75,76692,POINT (5.75000 36.75000)
1,36.75,6.25,76693,POINT (6.25000 36.75000)
2,35.75,-5.75,78109,POINT (-5.75000 35.75000)
3,35.75,7.25,78135,POINT (7.25000 35.75000)
4,35.25,-5.75,78829,POINT (-5.75000 35.25000)
...,...,...,...,...
5345,-34.25,20.25,178961,POINT (20.25000 -34.25000)
5346,-34.25,20.75,178962,POINT (20.75000 -34.25000)
5347,-34.25,21.25,178963,POINT (21.25000 -34.25000)
5348,-34.25,21.75,178964,POINT (21.75000 -34.25000)


What this shows is that the grid values are indexed to the 0.25

In [26]:
africa_mask = gpd.read_file('./synced-data/africa-mask.geojson')
africa_mask = africa_mask.to_crs(epsg=4326)

In [34]:
africa_bounds = africa_mask.bounds.values[0]
# indexing bounds to 0.25 values that AgMIP uses
africa_bounds[0] = np.floor(africa_bounds[0])-0.25
africa_bounds[1] = np.floor(africa_bounds[1])-0.25
africa_bounds[2] = np.ceil(africa_bounds[2])+0.25
africa_bounds[3] = np.ceil(africa_bounds[3])+0.25
print(africa_bounds)

[-26.25 -35.25  64.25  38.25]


In [35]:
# create a geodataframe of points within africa_bounds that are 0.5 apart
# this is the same as the AgMIP grid
x = np.arange(africa_bounds[0], africa_bounds[2], 0.5)
y = np.arange(africa_bounds[1], africa_bounds[3], 0.5)
xx, yy = np.meshgrid(x, y)
xx = xx.flatten()
yy = yy.flatten()
grid = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(xx, yy)])
grid = grid.set_crs(epsg=4326)

In [37]:
# drop all points from grid that are not in africa_mask
grid = grid[grid.within(africa_mask.geometry[0])]
grid.explore()

In [38]:
grid.to_file('./synced-data/empty-grid.geojson', driver='GeoJSON')
grid.to_csv('./synced-data/empty-grid.csv', index=False)